In [1]:
import pandas as pd
import numpy as np
from kamodo_ccmc.flythrough.utils import ConvertCoord
from sgp4.api import Satrec, jday, days2mdhms
from datetime import datetime, timezone, timedelta
import re
import json
import configparser
import requests
import time

class MyError(Exception):
    def __init___(self,args):
        Exception.__init__(self,"my exception was raised with arguments {0}".format(args))
        self.args = args

uriBase                = "https://www.space-track.org"
requestLogin           = "/ajaxauth/login"
requestCmdAction       = "/basicspacedata/query" 

config = configparser.ConfigParser()
config.read("./SpTrack-2.ini")
configUsr = config.get("configuration", "username")
configPwd = config.get("configuration", "password")
siteCred = {'identity': configUsr, 'password': configPwd}

In [2]:
%%time

norad_range = '<20000'
t1 = '2024-08-06'
t2 = '2024-08-22'

requestFindTLE = "/class/gp_history/NORAD_CAT_ID/" + norad_range + "/OBJECT_TYPE/debris/EPOCH/" + str(t1) + "--" + str(t2) + "/predicates/object_id,norad_cat_id,object_type,launch_date,decay_date,tle_line1,tle_line2/format/json/"

with requests.Session() as session:
    resp = session.post(uriBase + requestLogin, data = siteCred)
    if resp.status_code != 200:
        raise MyError(resp, "POST fail on login")

    resp = session.get(uriBase + requestCmdAction + requestFindTLE)
    if resp.status_code != 200:
        print(resp)
        raise MyError(resp, "GET fail on request for satellites")

    retData = json.loads(resp.text)   
session.close()

tle = []
for t in retData:
    tle = np.append(tle, [[t['TLE_LINE1']], [t['TLE_LINE2']]])

if len(tle)==0:
    raise Exception('No TLEs in search range.')

print('Number to process:', int(len(tle)/2))

v_list = []
for i in range(int(len(tle)/2)):
    d1 = 2000 + int(tle[2*i].split()[3][:2]) 
    d2, d3, t1, t2, t3 = days2mdhms(int(tle[2*i].split()[3][:2]),float(tle[2*i].split()[3][2:]))
    jd_deb = np.sum(jday(d1, d2, d3, t1, t2, t3))
    e, r, v = Satrec.twoline2rv(tle[2*i], tle[2*i+1]).sgp4(jd_deb,0)
    del e, r
    v_mag = np.sum(np.array(v)**2)**0.5
    v_list = np.append(v_list, v_mag)

print('Number processed:', len(v_list))
print('Max:', round(np.max(v_list),2), 'km/s')

Number to process: 64107
Number processed: 64107
Max: 9.88 km/s
CPU times: total: 5min 21s
Wall time: 5min 43s
